# Address Standardization and Normalization

## Install the required modules and import them

In [1]:
%pip install translitcodec
#%pip install codecs
%pip install pandas
%pip install numpy
%pip install number-parser
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import translitcodec
import codecs
from number_parser import parse
import pandas as pd
import numpy as np
import pyarrow
import re
import random
import os

## Load our addresses from pipe-delimited file and save as feather file

In [3]:
dataset_txt     = 'datasets/addresses.txt'
dataset_feather = 'datasets/addresses.feather'
if not os.path.exists(dataset_feather):
    pd.read_csv(dataset_txt, sep='|', index_col=False, na_filter=False)\
      .to_feather(dataset_feather)

## Load our feather file

In [5]:
df_source = pd.read_feather(dataset_feather)
df_source.shape
df_source

,streetAddress,city,stateAbbr,postcode
0,73379 Pfeffer Lodge Apt. 730,Giovannyfurt,CA,37698
1,2415 Adams Station Apt. 910,North Clydemouth,LA,35202
2,663 Feest Field,Wilmerchester,IL,77295-0882
3,41517 Gislason Plain Apt. 394,Juddville,TX,69059
4,5378 Alisa Walks,Corrinestad,NV,23062-7166
...,...,...,...,...
199,61148 Cheyanne Mount Apt. 801,West Chaya,TN,94553-9083
200,"757 Third Ave, Third Floor",New York,NY,10011
201,"757 3rd Avenue, 3rd Fl",New York,NY,10011
202,"757 3 Ave, Fl 3",New York,NY,10011


## use pandas vectorized string functions to clean text, and apply the transliteration
Afterwards, rename the columns with a mapping dictionary

In [6]:
column_mapping={
    'streetAddress':'address1',
    'city':'city',
    'stateAbbr':'state',
    'postcode':'postalcode',
}
def standardize_and_rename(df):
    return df.apply(lambda x: x.astype(str).str.strip().str.lower())\
                    .applymap(lambda x: codecs.encode(x,"translit/short"))\
                    .rename(columns=column_mapping)

### Given that our dataset has no countries, we'll just set them to random weighted countries

In [7]:
#---------------------------------------------------------------------------------------------------
def add_random_countries(df):
    """ Returns either the existing country series, or a series of random countries"""
    if 'country' in df:
        return df.country
    else:
        country_names   = (' ',  'united states','canada','england','germany')
        country_weights = (0.80, 0.10,            0.05,    0.03,     0.02)
        return random.choices(country_names, weights=country_weights, k=df.shape[0]) 
#---------------------------------------------------------------------------------------------------


### Normalize Countries
- Strip spaces, periods, parentheses
- Map to a country dictionary

In [8]:
#---------------------------------------------------------------------------------------------------
country_dict = {"afghanistan":"af","albania":"al","algeria":"dz","americansamoa":"as","andorra":"ad","angola":"ao","anguilla":"AI",
    "antarctica":"AQ","antigua":"AG","barbuda":"AG","antiguaandbarbuda":"AG","argentina":"AR","armenia":"AM","aruba":"AW",
    "australia":"AU","austria":"AT","azerbaijan":"AZ","bahamas":"BS","bahrain":"BH","bangladesh":"BD","barbados":"BB","belarus":"BY",
    "belgium":"BE","belize":"BZ","benin":"BJ","bermuda":"BM","bhutan":"BT","bolivia":"BO","boliviaplurinationalstateof":"BO",
    "bonaire":"BQ","sinteustatius":"BQ","saba":"BQ","bonaire, sinteustatiusandsaba":"BQ","bosnia":"BA","bosniaandherzegovina":"BA",
    "herzegovina":"BA","botswana":"BW","bouvetisland":"BV","brazil":"BR","britishindianoceanterritory":"IO","brunei":"BN",
    "bruneidarussalam":"BN","bulgaria":"BG","burkinafaso":"BF","burundi":"BI","cambodia":"KH","cameroon":"CM","canada":"CA",
    "capeverde":"CV","caymanislands":"KY","centralafricanrepublic":"CF","chad":"TD","chile":"CL","china":"CN","christmasisland":"CX",
    "cocosislands":"CC","keelingislands":"CC","cocoskeelingislands":"CC","colombia":"CO","comoros":"KM","congo":"CG",
    "congo, the democratic republic of the":"CD","cook islands":"CK","costa rica":"CR","cotedivoire":"CI","ivorycoast":"CI","croatia":"HR",
    "cuba":"CU","curacao":"CW","cyprus":"CY","czechrepublic":"CZ","denmark":"DK","djibouti":"DJ","dominica":"DM",
    "dominicanrepublic":"DO","ecuador":"EC","egypt":"EG","elalvador":"SV","equatorialguinea":"GQ","eritrea":"ER","estonia":"EE",
    "ethiopia":"ET","falklandislands":"FK","malvinas":"FK","falklandislandsmalvinas":"FK","faroeislands":"FO","fiji":"FJ",
    "finland":"FI","france":"FR","frenchguiana":"GF","frenchpolynesia":"PF","frenchsouthernterritories":"TF","gabon":"GA",
    "gambia":"GM","georgia":"GE","germany":"DE","ghana":"GH","gibraltar":"GI","greece":"GR","greenland":"GL","grenada":"GD",
    "guadeloupe":"GP","guam":"GU","guatemala":"GT","guernsey":"GG","guinea":"GN","guineabissau":"GW","guyana":"GY",
    "haiti":"HT","heardisland":"HM","mcdonaldislands":"HM","heardislandandmcdonaldislands":"HM","holysee":"VA",
    "vaticancitystate":"VA","vaticancity":"VA","holyseevaticancitystate":"VA","honduras":"HN","hongkong":"HK","hungary":"HU",
    "iceland":"IS","india":"IN","indonesia":"ID","iran":"IR","iranislamicrepublicof":"IR","iraq":"IQ","ireland":"IE","isleofman":"IM",
    "israel":"IL","italy":"IT","jamaica":"JM","japan":"JP","jersey":"JE","jordan":"JO","kazakhstan":"KZ","kenya":"KE","kiribati":"KI",
    "nkorea":"KP","northkorea":"KP","koreademocraticpeoplesrepublicof":"KP","korea":"KR","skorea":"KR","southkorea":"KR",
    "korearepublicof":"KR","kuwait":"KW","kyrgyzstan":"KG","lao":"LA","laopeoplesdemocraticrepublic":"LA","latvia":"LV",
    "lebanon":"LB","lesotho":"LS","liberia":"LR","libya":"LY","liechtenstein":"LI","lithuania":"LT","luxembourg":"LU","macau":"MO",
    "macao":"MO","macedonia":"MK","macedoniarepublicof":"MK","madagascar":"MG","malawi":"MW","malaysia":"MY","maldives":"MV",
    "mali":"ML","malta":"MT","marshallislands":"MH","martinique":"MQ","mauritania":"MR","mauritius":"MU","mayotte":"YT","mexico":"MX",
    "micronesia":"FM","micronesiafederatedstatesof":"FM","moldova":"MD","moldovarepublicof":"MD","monaco":"MC","mongolia":"MN",
    "montenegro":"ME","montserrat":"MS","morocco":"MA","mozambique":"MZ","myanmar":"MM","namibia":"NA","nauru":"NR","nepal":"NP",
    "nederlands":"NL","thenetherlands":"NL","holland":"NL","netherlands":"NL","newcaledonia":"NC","newzealand":"NZ","nicaragua":"NI",
    "niger":"NE","nigeria":"NG","niue":"NU","norfolkisland":"NF","nmarianaislands":"MP","northernmarianaislands":"MP","norway":"NO",
    "oman":"OM","pakistan":"PK","palau":"PW","palestine":"PS","palestinianterritory":"PS","palestinianterritoryoccupied":"PS",
    "panama":"PA","papuanewguinea":"PG","paraguay":"PY","peru":"PE","philippines":"PH","pitcairn":"PN","poland":"PL","portugal":"PT",
    "puertorico":"PR","qatar":"QA","reunion":"RE","romania":"RO","russia":"RU","russianfederation":"RU","rwanda":"RW",
    "stbarthelemy":"BL","saintbarthelemy":"BL","sainthelena":"SH","sthelena":"SH","ascension":"SH","tristandacunha":"SH",
    "sainthelenaascensionandtristandacunha":"SH","stkitts":"KN","saintkitts":"KN","nevis":"KN","saintkittsandnevis":"KN",
    "stlucia":"LC","saintlucia":"LC","saintmartin":"MF","stmartin":"MF","saintmartinfrenchpart":"MF","st pierre":"PM",
    "saintpierre":"PM","miquelon":"PM","saintpierreandmiquelon":"PM","stvincentandthegrenadines":"VC",
    "saintvincentandthegrenadines":"VC","samoa":"WS","sanmarino":"SM","saotomeandprincipe":"ST","saudiarabia":"SA","senegal":"SN",
    "serbia":"RS","seychelles":"SC","sierraleone":"SL","singapore":"SG","stmaarten":"SX","sintmaarten":"SX",
    "sintmaartendutchpart":"SX","slovakia":"SK","slovenia":"SI","solomonislands":"SB","somalia":"SO","safrica":"ZA",
    "southafrica":"ZA","sgeorgia":"GS","southgeorgia":"GS","ssandwichislands":"GS","southsandwichislands":"GS",
    "southgeorgiaandthesouthsandwichislands":"GS","spain":"ES","srilanka":"LK","sudan":"SD","suriname":"SR","ssudan":"SS",
    "southsudan":"SS","svalbard":"SJ","janmayen":"SJ","svalbardandjanmayen":"SJ","swaziland":"SZ","sweden":"SE","switzerland":"CH",
    "syria":"SY","syrian arab republic":"SY","taiwan":"TW","taiwan, province of china":"TW","tajikistan":"TJ","tanzania":"TZ",
    "tanzaniaunitedrepublicof":"TZ","thailand":"TH","timorleste":"TL","togo":"TG","tokelau":"TK","tonga":"TO","trinidad":"TT",
    "tobago":"TT","trinidadandtobago":"TT","tunisia":"TN","turkiye":"TR","turkey":"TR","turkmenistan":"TM",
    "turksandcaicosislands":"TC","tuvalu":"TV","uganda":"UG","ukraine":"UA","uae":"AE","unitedarabemirates":"AE",
    "unitedkingdom":"GB","uk":"GB","gbr":"uk","greatbritain":"GB","england":"GB","scotland":"GB","wales":"GB","northernireland":"GB",
    "nireland":"GB","unitedstates":"US","usa":"US","unitedstatesofamerica":"US","usminoroutlyingislands":"UM",
    "unitedstatesminoroutlyingislands":"UM","uruguay":"UY","uzbekistan":"UZ","vanuatu":"VU","venezuela":"VE",
    "venezuelabolivarianrepublicof":"VE","vietnam":"VN","britishvirginislands":"VG","bvi":"VG",
    "virginislandsbritish":"VG","usvirginislands":"VI","usvi":"VI","virginislands":"VI","wallisandfutuna":"WF",
    "yemen":"YE","zambia":"ZM","zimbabwe":"ZW"}
#---------------------------------------------------------------------------------------------------
# Remove commas, spaces, parentheses, periods, dashes, and apostrophes
country_translations = str.maketrans(r"",r"",r",. ()-'") 
default_country      = 'US'
#---------------------------------------------------------------------------------------------------

### Normalize States
- Remove periods, spaces
- Convert from dictionary

In [9]:
#---------------------------------------------------------------------------------------------------
state_translations = str.maketrans(r"",r"",r". ")
state_expansion={"al":"alabama","ak":"alaska","az":"arizona","ar":"arkansas","ca":"california","co":"colorado","ct":"connecticut","de":"delaware","fl":"florida","ga":"georgia","hi":"hawaii","id":"idaho","il":"illinois","in":"indiana","ia":"iowa","ks":"kansas","ky":"kentucky","la":"louisiana","me":"maine","md":"maryland","ma":"massachusetts","mi":"michigan","mn":"minnesota","ms":"mississippi","mo":"missouri","mt":"montana","ne":"nebraska","nv":"nevada","nh":"newhampshire","nj":"newjersey","nm":"newmexico","ny":"newyork","nc":"northcarolina","nd":"northdakota","oh":"ohio","ok":"oklahoma","or":"oregon","pa":"pennsylvania","ri":"rhodeisland","sc":"southcarolina","sd":"southdakota","tn":"tennessee","tx":"texas","ut":"utah","vt":"vermont","va":"virginia","wa":"washington","wv":"westvirginia","wi":"wisconsin","wy":"wyoming","dc":"districtofcolumbia","as":"americansamoa","gu":"guam","mp":"northernmarianaislands","pr":"puertorico","um":"unitedstatesminoroutlyingislands","vi":"usvirginislands","on":"ontario","qc":"quebec","ns":"novascotia","nb":"newbrunswick","mb":"manitoba","bc":"britishcolumbia","pe":"princeedwardisland","sk":"saskatchewan","ab":"alberta","nl":"newfoundlandandlabrador"}
state_contraction={"alabama":"al","alaska":"ak","arizona":"az","arkansas":"ar","california":"ca","colorado":"co","connecticut":"ct","delaware":"de","florida":"fl","georgia":"ga","hawaii":"hi","idaho":"id","illinois":"il","indiana":"in","iowa":"ia","kansas":"ks","kentucky":"ky","louisiana":"la","maine":"me","maryland":"md","massachusetts":"ma","michigan":"mi","minnesota":"mn","mississippi":"ms","missouri":"mo","montana":"mt","nebraska":"ne","nevada":"nv","newhampshire":"nh","newjersey":"nj","newmexico":"nm","newyork":"ny","northcarolina":"nc","northdakota":"nd","ohio":"oh","oklahoma":"ok","oregon":"or","pennsylvania":"pa","rhodeisland":"ri","southcarolina":"sc","southdakota":"sd","tennessee":"tn","texas":"tx","utah":"ut","vermont":"vt","virginia":"va","washington":"wa","westvirginia":"wv","wisconsin":"wi","wyoming":"wy","districtofcolumbia":"dc","americansamoa":"as","guam":"gu","northernmarianaislands":"mp","puertorico":"pr","unitedstatesminoroutlyingislands":"um","usvirginislands":"vi","ncarolina":"nc","ndakota":"nd","scarolina":"sc","sdakota":"sd","wvirginia":"wv","ontario":"on","quebec":"qc","novascotia":"ns","newbrunswick":"nb","manitoba":"mb","britishcolumbia":"bc","princeedwardisland":"pe","saskatchewan":"sk","alberta":"ab","newfoundlandandlabrador":"nl","qb":"qc","pq":"qc","newfoundland":"nl","labrador":"nl","ont":"on"}
state_stopwords=["the","and","of"]
#---------------------------------------------------------------------------------------------------

### Normalize postcodes
* strip spaces, dashes

In [10]:
#---------------------------------------------------------------------------------------------------
postcode_translations = str.maketrans(r"",r"",r"- ")
#---------------------------------------------------------------------------------------------------

### Normalize address lines
- Strip characters
- parse to convert number words to numbers
- regex to remove ordinals
- tokenize to words
- abbreviate
- remove stop words

In [11]:
#---------------------------------------------------------------------------------------------------
address_translations = str.maketrans(r"().,-:", r"      ", r"")
address_expansion={"allee":"alley","ally":"alley","aly":"alley","annex":"anex","annx":"anex","anx":"anex","arc":"arcade","av":"avenue","ave":"avenue","aven":"avenue","avenu":"avenue","avn":"avenue","avnue":"avenue","bayoo":"bayou","bch":"beach","bnd":"bend","blf":"bluff","bluf":"bluff","bot":"bottom","btm":"bottom","bottm":"bottom","blvd":"boulevard","boul":"boulevard","boulv":"boulevard","br":"branch","brnch":"branch","brdge":"bridge","brg":"bridge","brk":"brook","byp":"bypass","bypa":"bypass","bypas":"bypass","byps":"bypass","cp":"camp","cmp":"camp","canyn":"canyon","cnyn":"canyon","cpe":"cape","causwa":"causeway","cswy":"causeway","cen":"center","cent":"center","centr":"center","centre":"center","cnter":"center","cntr":"center","ctr":"center","cir":"circle","circ":"circle","circl":"circle","crcl":"circle","crcle":"circle","clf":"cliff","clfs":"cliffs","clb":"club","cor":"corner","cors":"corners","crse":"course","ct":"court","cts":"courts","cv":"cove","crk":"creek","cres":"crescent","crsent":"crescent","crsnt":"crescent","crssng":"crossing","xing":"crossing","dl":"dale","dm":"dam","div":"divide","dv":"divide","dvd":"divide","dr":"drive","driv":"drive","drv":"drive","est":"estate","ests":"estates","exp":"expressway","expr":"expressway","express":"expressway","expw":"expressway","expy":"expressway","ext":"extension","extn":"extension","extnsn":"extension","exts":"extensions","fls":"falls","frry":"ferry","fry":"ferry","fld":"field","flds":"fields","flt":"flat","flts":"flats","frd":"ford","forests":"forest","frst":"forest","forg":"forge","frg":"forge","frk":"fork","frks":"forks","frt":"fort","ft":"fort","freewy":"freeway","frway":"freeway","frwy":"freeway","fwy":"freeway","gardn":"garden","grden":"garden","grdn":"garden","gdns":"gardens","grdns":"gardens","gatewy":"gateway","gatway":"gateway","gtway":"gateway","gtwy":"gateway","gln":"glen","grn":"green","grov":"grove","grv":"grove","harb":"harbor","harbr":"harbor","hbr":"harbor","hrbor":"harbor","hvn":"haven","ht":"heights","hts":"heights","highwy":"highway","hiway":"highway","hiwy":"highway","hway":"highway","hwy":"highway","hl":"hill","hls":"hills","hllw":"hollow","hollows":"hollow","holw":"hollow","holws":"hollow","inlt":"inlet","is":"island","islnd":"island","islnds":"islands","iss":"islands","isles":"isle","jct":"junction","jction":"junction","jctn":"junction","junctn":"junction","juncton":"junction","jctns":"junctions","jcts":"junctions","ky":"key","kys":"keys","knl":"knoll","knol":"knoll","knls":"knolls","lk":"lake","lks":"lakes","lndg":"landing","lndng":"landing","ln":"lane","lgt":"light","lf":"loaf","lck":"lock","lcks":"locks","ldg":"lodge","ldge":"lodge","lodg":"lodge","loops":"loop","mnr":"manor","mnrs":"manors","mdw":"meadows","mdws":"meadows","medows":"meadows","missn":"mission","mssn":"mission","mnt":"mount","mt":"mount","mntain":"mountain","mntn":"mountain","mountin":"mountain","mtin":"mountain","mtn":"mountain","mntns":"mountains","nck":"neck","orch":"orchard","orchrd":"orchard","ovl":"oval","prk":"park","parkwy":"parkway","pkway":"parkway","pkwy":"parkway","pky":"parkway","pkwys":"parkways","paths":"path","pikes":"pike","pnes":"pines","pl":"place","pln":"plain","plns":"plains","plz":"plaza","plza":"plaza","pt":"point","pts":"points","prt":"port","prts":"ports","pr":"prairie","prr":"prairie","rad":"radial","radiel":"radial","radl":"radial","ranches":"ranch","rnch":"ranch","rnchs":"ranch","rpd":"rapid","rpds":"rapids","rst":"rest","rdg":"ridge","rdge":"ridge","rdgs":"ridges","riv":"river","rvr":"river","rivr":"river","rd":"road","rds":"roads","shl":"shoal","shls":"shoals","shoar":"shore","shr":"shore","shoars":"shores","shrs":"shores","spg":"spring","spng":"spring","sprng":"spring","spgs":"springs","spngs":"springs","sprngs":"springs","sq":"square","sqr":"square","sqre":"square","squ":"square","sqrs":"squares","sta":"station","statn":"station","stn":"station","stra":"stravenue","strav":"stravenue","straven":"stravenue","stravn":"stravenue","strvn":"stravenue","strvnue":"stravenue","streme":"stream","strm":"stream","strt":"street","st":"street","str":"street","smt":"summit","sumit":"summit","sumitt":"summit","ter":"terrace","terr":"terrace","traces":"trace","trce":"trace","tracks":"track","trak":"track","trk":"track","trks":"track","trails":"trail","trl":"trail","trls":"trail","trlr":"trailer","trlrs":"trailer","tunel":"tunnel","tunl":"tunnel","tunls":"tunnel","tunnels":"tunnel","tunnl":"tunnel","trnpk":"turnpike","turnpk":"turnpike","un":"union","vally":"valley","vlly":"valley","vly":"valley","vlys":"valleys","vdct":"viaduct","via":"viaduct","viadct":"viaduct","vw":"view","vws":"views","vill":"village","villag":"village","villg":"village","villiage":"village","vlg":"village","vlgs":"villages","vl":"ville","vis":"vista","vist":"vista","vst":"vista","vsta":"vista","wy":"way","wls":"wells","apt":"apartment","bsmt":"basement","bldg":"building","dept":"department","fl":"floor","frnt":"front","hngr":"hanger","lbby":"lobby","lowr":"lower","ofc":"office","ph":"penthouse","rm":"room","spc":"space","ste":"suite","trlr":"trailer","uppr":"upper","n":"north","s":"south","e":"east","w":"west","ne":"northeast","nw":"northwest","se":"southeast","sw":"southwest","1":"first","2":"second","3":"third","4":"fourth","5":"fifth","6":"sixth","7":"seventh","8":"eighth","9":"ninth","10":"tenth","11":"eleventh","12":"twelfth","13":"thirteenth"}
address_contraction={"allee":"aly","alley":"aly","ally":"aly","anex":"anx","annex":"anx","annx":"anx","arcade":"arc","av":"ave","aven":"ave","avenu":"ave","avenue":"ave","avn":"ave","avnue":"ave","bayoo":"byu","bayou":"byu","beach":"bch","bend":"bnd","bluf":"blf","bluff":"blf","bluffs":"blfs","bot":"btm","bottm":"btm","bottom":"btm","boul":"blvd","boulevard":"blvd","boulv":"blvd","brnch":"br","branch":"br","brdge":"brg","bridge":"brg","brook":"brk","brooks":"brks","burg":"bg","burgs":"bgs","bypa":"byp","bypas":"byp","bypass":"byp","byps":"byp","camp":"cp","cmp":"cp","canyn":"cyn","canyon":"cyn","cnyn":"cyn","cape":"cpe","causeway":"cswy","causwa":"cswy","cen":"ctr","cent":"ctr","center":"ctr","centr":"ctr","centre":"ctr","cnter":"ctr","cntr":"ctr","centers":"ctrs","circ":"cir","circl":"cir","circle":"cir","crcl":"cir","crcle":"cir","circles":"cirs","cliff":"clf","cliffs":"clfs","club":"clb","common":"cmn","commons":"cmns","corner":"cor","corners":"cors","course":"crse","court":"ct","courts":"cts","cove":"cv","coves":"cvs","creek":"crk","crescent":"cres","crsent":"cres","crsnt":"cres","crest":"crst","crossing":"xing","crssng":"xing","crossroad":"xrd","crossroads":"xrds","curve":"curv","dale":"dl","dam":"dm","div":"dv","divide":"dv","dvd":"dv","driv":"dr","drive":"dr","drv":"dr","drives":"drs","estate":"est","estates":"ests","exp":"expy","expr":"expy","express":"expy","expressway":"expy","expw":"expy","extension":"ext","extn":"ext","extnsn":"ext","falls":"fls","ferry":"fry","frry":"fry","field":"fld","fields":"flds","flat":"flt","flats":"flts","ford":"frd","fords":"frds","forest":"frst","forests":"frst","forg":"frg","forge":"frg","forges":"frgs","fork":"frk","forks":"frks","fort":"ft","frt":"ft","freeway":"fwy","freewy":"fwy","frway":"fwy","frwy":"fwy","garden":"gdn","gardn":"gdn","grden":"gdn","grdn":"gdn","gardens":"gdns","grdns":"gdns","gateway":"gtwy","gatewy":"gtwy","gatway":"gtwy","gtway":"gtwy","glen":"gln","glens":"glns","green":"grn","greens":"grns","grov":"grv","grove":"grv","groves":"grvs","harb":"hbr","harbor":"hbr","harbr":"hbr","hrbor":"hbr","harbors":"hbrs","haven":"hvn","ht":"hts","highway":"hwy","highwy":"hwy","hiway":"hwy","hiwy":"hwy","hway":"hwy","hill":"hl","hills":"hls","hllw":"holw","hollow":"holw","hollows":"holw","holws":"holw","island":"is","islnd":"is","islands":"iss","islnds":"iss","isles":"isle","jction":"jct","jctn":"jct","junction":"jct","junctn":"jct","juncton":"jct","jctns":"jcts","junctions":"jcts","key":"ky","keys":"kys","knol":"knl","knoll":"knl","knolls":"knls","lake":"lk","lakes":"lks","landing":"lndg","lndng":"lndg","lane":"ln","light":"lgt","lights":"lgts","loaf":"lf","lock":"lck","locks":"lcks","ldge":"ldg","lodg":"ldg","lodge":"ldg","loops":"loop","manor":"mnr","manors":"mnrs","meadow":"mdw","mdw":"mdws","meadows":"mdws","medows":"mdws","mill":"ml","mills":"mls","missn":"msn","mssn":"msn","motorway":"mtwy","mnt":"mt","mount":"mt","mntain":"mtn","mntn":"mtn","mountain":"mtn","mountin":"mtn","mtin":"mtn","mntns":"mtns","mountains":"mtns","neck":"nck","orchard":"orch","orchrd":"orch","ovl":"oval","overpass":"opas","prk":"park","parks":"park","parkway":"pkwy","parkwy":"pkwy","pkway":"pkwy","pky":"pkwy","parkways":"pkwy","pkwys":"pkwy","passage":"psge","paths":"path","pikes":"pike","pine":"pne","pines":"pnes","plain":"pln","plains":"plns","plaza":"plz","plza":"plz","point":"pt","points":"pts","port":"prt","ports":"prts","prairie":"pr","prr":"pr","rad":"radl","radial":"radl","radiel":"radl","ranch":"rnch","ranches":"rnch","rnchs":"rnch","rapid":"rpd","rapids":"rpds","rest":"rst","rdge":"rdg","ridge":"rdg","ridges":"rdgs","river":"riv","rvr":"riv","rivr":"riv","road":"rd","roads":"rds","route":"rte","shoal":"shl","shoals":"shls","shoar":"shr","shore":"shr","shoars":"shrs","shores":"shrs","skyway":"skwy","spng":"spg","spring":"spg","sprng":"spg","spngs":"spgs","springs":"spgs","sprngs":"spgs","spurs":"spur","sqr":"sq","sqre":"sq","squ":"sq","square":"sq","sqrs":"sqs","squares":"sqs","station":"sta","statn":"sta","stn":"sta","strav":"stra","straven":"stra","stravenue":"stra","stravn":"stra","strvn":"stra","strvnue":"stra","stream":"strm","streme":"strm","street":"st","strt":"st","str":"st","streets":"sts","sumit":"smt","sumitt":"smt","summit":"smt","terr":"ter","terrace":"ter","throughway":"trwy","trace":"trce","traces":"trce","track":"trak","tracks":"trak","trk":"trak","trks":"trak","trafficway":"trfy","trail":"trl","trails":"trl","trls":"trl","trailer":"trlr","trlrs":"trlr","tunel":"tunl","tunls":"tunl","tunnel":"tunl","tunnels":"tunl","tunnl":"tunl","trnpk":"tpke","turnpike":"tpke","turnpk":"tpke","underpass":"upas","union":"un","unions":"uns","valley":"vly","vally":"vly","vlly":"vly","valleys":"vlys","vdct":"via","viadct":"via","viaduct":"via","view":"vw","views":"vws","vill":"vlg","villag":"vlg","village":"vlg","villg":"vlg","villiage":"vlg","villages":"vlgs","ville":"vl","vist":"vis","vista":"vis","vst":"vis","vsta":"vis","walks":"walk","wy":"way","well":"wl","wells":"wls","apartment":"apt","basement":"bsmt","building":"bldg","department":"dept","floor":"fl","front":"frnt","hanger":"hngr","lobby":"lbby","lower":"lowr","office":"ofc","penthouse":"ph","room":"rm","space":"spc","suite":"ste","trailer":"trlr","upper":"uppr","north":"n","south":"s","east":"e","west":"w","northeast":"ne","northwest":"nw","southeast":"se","southwest":"sw","first":"1","second":"2","third":"3","fourth":"4","fifth":"5","sixth":"6","seventh":"7","eighth":"8","ninth":"9","tenth":"10","eleventh":"11","twelfth":"12","thirteenth":"13"}
address_stopwords=["the","and","of","c/o","attention","attn"]
#---------------------------------------------------------------------------------------------------
def Normalize_Address_Line( address ):
    """ Normalization for address lines: removes punctuation, converts words to numbers, tokenizes, abbreviates, and removes stopwords """
    tokens = re.sub(r"(?<=[0-9])(?:st|nd|rd|th)","",address.translate( address_translations ),0, re.MULTILINE ).split()
    tokens = [address_contraction.get(token, token) for token in tokens] ## abbreviate
    tokens = [address_expansion.get(token, token) for token in tokens] ## expand
    tokens = [token for token in tokens if token not in address_stopwords] ## remove stopwords!
    return ' '.join(tokens)
#---------------------------------------------------------------------------------------------------

### Normalize Cities
- strip charactors
- tokenize to words
- abbreviate
- remove stop words

In [12]:

#---------------------------------------------------------------------------------------------------
city_translations = str.maketrans(r".,", r"  ", r"-")
city_expansion={"n":"north","e":"east","s":"south","w":"west","mt":"mount","lk":"lake","st":"saint","ft":"fort","spgs":"springs","fls":"falls","bch":"beach"}
city_contraction={"north":"n","east":"e","south":"s","west":"w","mount":"mt","lake":"lk","saint":"st","fort":"ft","springs":"spgs","falls":"fls","beach":"bch"}
city_stopwords=["the","and","of"]
#---------------------------------------------------------------------------------------------------
def Normalize_City_Line( city ):
    """ Normalization for address lines: removes punctuation, converts words to numbers, tokenizes, abbreviates, and removes stopwords """
    tokens = city.translate(city_translations).split()
    tokens = [city_contraction.get(token, token) for token in tokens] ## abbreviate
    tokens = [city_expansion.get(token, token) for token in tokens] ## expand
    tokens = [token for token in tokens if token not in city_stopwords] ## remove stopwords
    return ' '.join(tokens)
#---------------------------------------------------------------------------------------------------

### Normalize Personal names and Corporations
- strip characters
- tokenize to words
- remove stop words

In [13]:
### Normalize Personal names and Corporations
#---------------------------------------------------------------------------------------------------
name_translations = str.maketrans(r".,", r"  ", r"-")
name_stopwords=["the","and","of","c/o","attention","attn","dr","prof"]
name_expansion={"dr":"doctor","prof":"professor"}
name_contraction={"doctor":"dr","professor":"prof"}
#---------------------------------------------------------------------------------------------------
def Normalize_Names( name ):
    """ Normalization for names: removes punctuation, converts words to numbers, tokenizes, abbreviates, and removes stopwords """
    tokens = name.translate(name_translations).split()
    tokens = [name_contraction.get(token, token) for token in tokens] ## abbreviate
    tokens = [name_expansion.get(token, token) for token in tokens] ## expand
    tokens = [token for token in tokens if token not in name_stopwords] ## remove stopwords
    return ' '.join(tokens)   
#---------------------------------------------------------------------------------------------------


### Now make our normalized dataset

In [14]:
#%%timeit

df_source = pd.read_feather(dataset_feather)
df2 = standardize_and_rename(df_source)
df2['country'] = add_random_countries(df2)
# 4.47 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) (204)
# 4.44 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
#%%timeit

df2.address1   = df2.address1.apply(Normalize_Address_Line)
df2.city       = df2.city.apply(Normalize_City_Line)
df2.state      = df2.state.apply(lambda x: state_contraction.get(x.translate(state_translations),x))\
                          .apply(lambda x: state_expansion.get(x.translate(state_translations),x))
df2.postalcode = df2.postalcode.apply(lambda x: x.translate(postcode_translations))
df2.country    = df2.country.apply(lambda x: country_dict.get(x.translate(country_translations),x))\
                            .apply(lambda x: (default_country if x.strip()=='' else x).upper())
## 3.31 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each) (204)
# 98 ms ± 45.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# 2.07 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# 2.01 ms ± 41.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
# 2.13 ms ± 164 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

In [16]:
df_source

,streetAddress,city,stateAbbr,postcode
0,73379 Pfeffer Lodge Apt. 730,Giovannyfurt,CA,37698
1,2415 Adams Station Apt. 910,North Clydemouth,LA,35202
2,663 Feest Field,Wilmerchester,IL,77295-0882
3,41517 Gislason Plain Apt. 394,Juddville,TX,69059
4,5378 Alisa Walks,Corrinestad,NV,23062-7166
...,...,...,...,...
199,61148 Cheyanne Mount Apt. 801,West Chaya,TN,94553-9083
200,"757 Third Ave, Third Floor",New York,NY,10011
201,"757 3rd Avenue, 3rd Fl",New York,NY,10011
202,"757 3 Ave, Fl 3",New York,NY,10011


In [17]:
df2

,address1,city,state,postalcode,country
0,73379 pfeffer lodge apartment 730,giovannyfurt,california,37698,US
1,2415 adams station apartment 910,north clydemouth,louisiana,35202,US
2,663 feest field,wilmerchester,illinois,772950882,US
3,41517 gislason plain apartment 394,juddville,texas,69059,US
4,5378 alisa walk,corrinestad,nevada,230627166,US
...,...,...,...,...,...
199,61148 cheyanne mount apartment 801,west chaya,tennessee,945539083,US
200,757 third avenue third floor,new york,newyork,10011,US
201,757 third avenue third floor,new york,newyork,10011,US
202,757 third avenue floor third,new york,newyork,10011,US
